Take input goals file and create a list of results.

In [1]:
import pandas as pd

In [2]:
goles = pd.read_csv(
    'input/goles - liga de campeones - Sheet1.tsv',
    sep='\t',
    dtype=str
)

In [3]:
goles.head()

,season,round,tie,game,minute,away,extra,shootout,note
0,2017,first,city-monaco,1,26,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,32,a,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,40,a,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,58,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,61,a,NaN,NaN,NaN


In [4]:
goles['season'].fillna(method='pad', inplace=True)
goles['round'].fillna(method='pad', inplace=True)
goles['tie'].fillna(method='pad', inplace=True)
goles['game'].fillna(method='pad', inplace=True)

In [5]:
goles['t1'] = goles.tie.apply(lambda x: x.split('-')[0])
goles['t2'] = goles.tie.apply(lambda x: x.split('-')[1])

In [6]:
goles.head()

,season,round,tie,game,minute,away,extra,shootout,note,t1,t2
0,2017,first,city-monaco,1,26,NaN,NaN,NaN,NaN,city,monaco
1,2017,first,city-monaco,1,32,a,NaN,NaN,NaN,city,monaco
2,2017,first,city-monaco,1,40,a,NaN,NaN,NaN,city,monaco
3,2017,first,city-monaco,1,58,NaN,NaN,NaN,NaN,city,monaco
4,2017,first,city-monaco,1,61,a,NaN,NaN,NaN,city,monaco


In [7]:
def sum_goals(part):
    '''
    Gets the goals and away goals for a part of a game.
    A "part" is regular or extra time.
    '''
    t1g =\
    len(part[(part.game=='1') & (part.away.isnull())]) +\
    len(part[(part.game=='2') & (part.away.notnull())])
    
    t2g =\
    len(part[(part.game=='1') & (part.away.notnull())]) +\
    len(part[(part.game=='2') & (part.away.isnull())])
    
    t1ag = len(part[(part.game=='2') & (part.away.notnull())])
    t2ag = len(part[(part.game=='1') & (part.away.notnull())])
    
    return t1g, t2g, t1ag, t2ag

In [8]:
def get_tie_result(goals):
    winner = None
    pk = agr = aet = False
    result = ""
    
    t1 = goals.t1.unique()[0]
    t2 = goals.t2.unique()[0] 
    
    reg = goals[
        (goals.minute != 'pk') &
        (goals.extra.isnull()) &
        (goals.minute.notnull())
    ]
    
    et = goals[
        (goals.minute != 'pk') &
        (goals.extra.notnull()) &
        (goals.minute.notnull())
    ]
    
    pkg = goals[
        goals.minute == 'pk'
    ]
    
    # goals and away goals in regulation
    t1g, t2g, t1ag, t2ag = sum_goals(reg)
    
    # goals and away goals in extra time
    t1etg, t2etg, t1aetg, t2aetg = sum_goals(et)
    
    # pk shootout result
    t1pkwin = len(pkg[pkg.away == 'a'])
    
    # if goal sums differ
    if (t1g+t1etg) != (t2g+t2etg):
        # outright win
        winner = t1 if t1g > t2g else t2
    else:
        # if away goal sums differ
        if (t1ag+t1aetg) != (t2ag+t2aetg):
            # away goals win
            winner = t1 if t1ag > t2ag else t2
            agr = True
        else:
            # pk shootout
            winner = t1 if t1pkwin else t2
            pk = True
    
    # if goals in regulation are tied 
    # and away goals in regulation are tied
    # this went to extra time
    if (t1g == t2g) & (t1ag == t2ag):
        aet = True
    
    # a string summarizing the result
    result = "{} ({}-{}{}{}{}".format(
        t1,
        t1g+t1etg,
        t2g+t2etg,
        " aet) " if aet else ") ",
        t2,
        ", {} won on penalties".format(winner) if pk else (
            ", {} won on away goals".format(winner) if agr else ""
        )
    )
    
    return pd.Series(
        [winner, pk, agr, aet, result],
        index=['winner', 'pk', 'agr', 'aet', 'result']
    )

In [9]:
results = goles.groupby(['season','round','tie']).apply(get_tie_result)

In [10]:
results

winner     pk    agr    aet  \
season round tie                                                        
2008   first arsenal-milan               arsenal  False  False  False   
             celtic-barcelona          barcelona  False  False  False   
             fenerbahce-sevilla       fenerbahce   True  False   True   
             liverpool-inter           liverpool  False  False  False   
             lyon-united                  united  False  False  False   
             olympiacos-chelsea          chelsea  False  False  False   
             roma-madrid                    roma  False  False  False   
             schalke-porto               schalke   True  False   True   
       qtr   arsenal-liverpool         liverpool  False  False  False   
             fenerbahce-chelsea          chelsea  False  False  False   
             roma-united                  united  False  False  False   
             schalke-barcelona         barcelona  False  False  False   
       semi  barcelona-united             united  False  False  False   
             liverpool-chelsea           chelsea  False  False   True   
2009   first arsenal-roma                arsenal   True  False   True   
             atletico-porto                porto  False   True  False   
             chelsea-juventus            chelsea  False  False  False   
             inter-united                 united  False  False  False   
             lyon-barcelona            barcelona  False  False  False   
             madrid-liverpool          liverpool  False  False  False   
             sporting-bayern              bayern  False  False  False   
             villareal-panathinaikos   villareal  False  False  False   
       qtr   barcelona-bayern          barcelona  False  False  False   
             liverpool-chelsea           chelsea  False  False  False   
             united-porto                 united  False  False  False   
             villareal-arsenal           arsenal  False  False  False   
       semi  barcelona-chelsea         barcelona  False   True  False   
             united-arsenal               united  False  False  False   
2010   first bayern-fiorentina            bayern  False   True  False   
             cska-sevilla                   cska  False  False  False   
...                                          ...    ...    ...    ...   
2017   first psg-barcelona             barcelona  False  False  False   
             sevilla-leicester         leicester  False  False  False   
       qtr   atletico-leicester         atletico  False  False  False   
             bayern-madrid                madrid  False  False   True   
             dortmund-monaco              monaco  False  False  False   
             juventus-barcelona         juventus  False  False  False   
       semi  madrid-atletico              madrid  False  False  False   
             monaco-juventus            juventus  False  False  False   
2018   first basel-city                     city  False  False  False   
             bayern-besiktas              bayern  False  False  False   
             chelsea-barcelona         barcelona  False  False  False   
             juventus-tottenham         juventus  False  False  False   
             madrid-psg                   madrid  False  False  False   
             porto-liverpool           liverpool  False  False  False   
             sevilla-united              sevilla  False  False  False   
             shakhtar-roma              shakhtar  False  False  False   
       qtr   barcelona-roma                 roma  False   True  False   
             juventus-madrid              madrid  False  False  False   
             liverpool-city            liverpool  False  False  False   
             sevilla-bayern               bayern  False  False  False   
       semi  bayern-madrid                madrid  False  False  False   
             liverpool-roma            liverpool  False  False  False   
2019   first ajax-madrid   

In [11]:
results.to_csv('output/results.csv')